In [3]:
from PyQt5.QtWidgets import QDialog, QVBoxLayout, QLabel, QPushButton

class InfoWindow(QDialog):
    def __init__(self, pet):
        super().__init__()

        self.pet = pet
        self.setWindowTitle(f"{pet.name.capitalize()} — Info")
        self.setMinimumWidth(250)
        self.setStyleSheet(self.info_style())

        self.build_ui()

        self.update_timer = QTimer(self)
        self.update_timer.timeout.connect(self.update_info)
        self.update_timer.start(100)
    
    def build_ui(self):
        layout = QVBoxLayout()

        self.name_label = QLabel()
        self.level_label = QLabel()
        self.evo_label = QLabel()

        layout.addWidget(self.name_label)
        layout.addWidget(self.level_label)
        layout.addWidget(self.evo_label)

        self.setLayout(layout)

        self.update_info()

    def info_style(self):
        with open("info_window.qss", "r") as f:
            style = f.read()

        return style

    def update_info(self):
        self.name_label.setText(f"Nome: {self.pet.name}")
        self.level_label.setText(f"Nível: {self.pet.level}")
        self.evo_label.setText(f"Evolução: {self.pet.evolution_stage}")

In [4]:
import sys
import random
import math
import os
from PyQt5.QtWidgets import QApplication, QLabel, QMenu, QAction
from PyQt5.QtGui import QPixmap, QPainter, QColor, QTransform
from PyQt5.QtCore import Qt, QTimer

class Pet(QLabel):
    def __init__(self, name):
        super().__init__()

        # image related variables
        self.name = name
        self.evolution_stage = 0
        # base_dir = os.path.dirname(os.path.abspath(__file__))
        # self.img_dir = os.path.join(base_dir, "../imgs")
        # self.image_path = os.path.join(
        #     self.img_dir, f"{self.name}_{self.evolution_stage}.png"
        # )
        self.image_path = f"../imgs/{self.name}_{self.evolution_stage}.png"

        # Pet state
        self.pos_x, self.pos_y = random.randint(500, 1500), random.randint(200, 400)
        self.direction = random.choice([1, -1])
        self.vx, self.vy = random.randint(0, 3)*self.direction, 0
        self.level = 5
        self.walk_cycle = 0
        self.manager = None

        # Flags / Utility
        self.drag_offset = None
        self.last_mouse_pos = None
        self.on_delay = False
        self.is_walking = False
        self.in_battle = False

        # Initial configuration
        self._load_image()
        self._setup_window()
        self._setup_timers()
        self._setup_screens()
        self._create_context_menu()

    # ========== Setup ==========

    def _load_image(self):
        self.original_pixmap = QPixmap(self.image_path)
        pixmap = self.original_pixmap
        
        if self.direction == 1:
            transform = QTransform()
            transform.scale(-1, 1)
            pixmap = pixmap.transformed(transform)
        
        self.setPixmap(pixmap)

    def _setup_window(self):
        self.setGeometry(self.pos_x, self.pos_y, 128, 128)
        # Unique window title for each pet
        self.setWindowTitle(f"Pet - {self.name.capitalize()}")
        self.setWindowFlags(
            Qt.FramelessWindowHint
            | Qt.WindowStaysOnTopHint
            | Qt.Tool
        )
        self.setAttribute(Qt.WA_TranslucentBackground)
        self.setAttribute(Qt.WA_AlwaysStackOnTop)

    def _setup_timers(self):
        self.timer = QTimer()
        self.timer.timeout.connect(self._move_pet)
        self.timer.start(15)
        self.delay_timer = QTimer()
        self.delay_timer.timeout.connect(self._end_delay)

    def _setup_screens(self):
        self.screens = QApplication.instance().screens()
        area = self.screens[0].virtualGeometry()
        self.left, self.top = area.left(), area.top()
        self.right = area.right() - self.width()
        self.floor = area.bottom() - self.height() - 20

    def _create_context_menu(self):

        # carrega a QSS uma vez
        try:
            with open("menu.qss", "r") as f:
                menu_style = f.read()
        except FileNotFoundError:
            menu_style = ""  # fallback para não quebrar
    
        # cria o menu e guarda como atributo
        self.context_menu = QMenu(self)
        self.context_menu.setStyleSheet(menu_style)
    
        # cria ações
        self.info_action = QAction("[   ⓘ   Info   ]", self)
        self.close_action = QAction("[   ✖   Close  ]", self)
    
        # conecta ações
        self.info_action.triggered.connect(self.show_info)
        self.close_action.triggered.connect(self.close_pet)
    
        # adiciona no menu
        self.context_menu.addAction(self.info_action)
        self.context_menu.addSeparator()
        self.context_menu.addAction(self.close_action)

    # ========== Movement ==========

    def _move_pet(self):
        """
        Main movement loop.
        """

        if self.drag_offset:
            return

        if not self._fall_pet():
            self._walk_pet()

            if self.level >= 15 and self.evolution_stage == 0:
                self._evolve_pet()

            if self.level >= 36 and self.evolution_stage == 1:
                self._evolve_pet()

        self._cant_escape_bounds()
        self.move(self.pos_x, self.pos_y)

    def _fall_pet(self):
        """
        Simulates gravity.

        Returns:
            if the pet is falling (bool).
        """

        if self.is_walking:
            return False

        # If the pet is above the floor level, it will begin to fall.
        if self.pos_y < self.floor:

            # Its vertical speed will increase with each iteration, until it reaches terminal velocity.
            self.vy = min(self.vy + 1, 50)
            self.pos_y += self.vy

            # Its horizontal speed remains the same, to simulate inertia.
            self.vx = min(self.vx, 50)
            self.pos_x += self.vx

            return True

        else:
            self.is_walking = True
            return False

    def _walk_pet(self):
        """
        Makes the pet walk.
        """

        if not self.is_walking or self.on_delay or self.in_battle:
            return

        self.walk_cycle += 1
        self.vx = 2 * self.direction
        self.pos_x += self.vx

        # Smooth vertical motion (sinusoidal rocking)
        offset_y = 5 * math.sin(self.walk_cycle * 0.5)
        self.pos_y = math.floor(self.floor + offset_y)

        # Chance to change directions
        if random.random() < 0.005:
            self.direction *= -1
            self.vx *= -1
            self._flip_image()

        # Chance to pause (delay)
        elif random.random() < 0.002:
            self._start_delay()

        # Chance to jump
        elif random.random() < 0.001:
            self._jump_pet()

    def _jump_pet(self, min_height: int = -30, max_height: int = -15):
        """
        Makes the pet jump.

        Parameters
            min_height (int): minimum value.
            max_height (int): maximum value.
        """

        self.is_walking = False

        # Adds a random vertical speed to the pet.
        self.vy = random.randint(min_height, max_height)
        self.pos_y += self.vy

    def _cant_escape_bounds(self):
        """
        Prevents the pet from escaping the screen bounds.
        """

        # Floor is determined by the bottom bound of which screen the pet is in.
        bounds = [screen.geometry() for screen in self.screens]
        for bound in bounds:
            if self.pos_x >= bound.left() and self.pos_x <= bound.right():
                self.floor = bound.bottom() - self.height() - 20
                break

        # If beyond any of the limits, the pet is obstructed to go any further.
        self.pos_x = max(self.left, min(self.pos_x, self.right))
        self.pos_y = max(self.top, min(self.pos_y, self.floor))

    def _evolve_pet(self):
        """
        Evolves the pet.
        """

        iteration_counter = 0
        # The sprite is whiten to simulate evolution.
        self._color_image(r=255, g=255, b=255, alpha=200)

        def animate():
            nonlocal iteration_counter
            iteration_counter += 1

            # When the evolution is over
            if iteration_counter >= 300:
                # Changes the pet sprite.
                self.evolution_stage += 1
                # self.image_path = os.path.join(
                #     self.img_dir, f"{self.name}_{self.evolution_stage}.png"
                # )
                self.image_path = f"../imgs/{self.name}_{self.evolution_stage}.png"
                self._load_image()

                # The original timer is run again.
                self.reset_timer(callback=self._move_pet, interval=15)

        # Starts a timer for the pet to evolve.
        self.reset_timer(callback=animate, interval=15)

    def lose_battle(self):
        """
        Does the animation when losing a battle.
        """

        self._jump_pet(min_height=-20, max_height=-10)
        self.vx = random.randint(10, 15) * -self.direction

    def win_battle(self):
        """
        Does the animation when winning a battle.
        """
        self._jump_pet(min_height=-10, max_height=-10)

    # ========== Visual effects ==========

    def _flip_image(self):
        """
        Flips the image on the horizontal axis.
        """
        transform = QTransform()
        transform.scale(-1, 1)
        flipped_pixmap = self.pixmap().transformed(transform)
        self.setPixmap(flipped_pixmap)

    def _color_image(self, r: int, g: int, b: int, alpha: int = 200):
        """
        Colors the image.

        Parameters:
            alpha (int): Transparency.
            r (int): Color red.
            g (int): Color green.
            b (int): Color blue.
        """
        pixmap = self.pixmap().copy()
        painter = QPainter(pixmap)
        painter.setCompositionMode(QPainter.CompositionMode_SourceIn)
        painter.fillRect(pixmap.rect(), QColor(r, g, b, alpha))
        painter.end()
        self.setPixmap(pixmap)

    # ========== Mouse events ==========

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drag_offset = event.pos()  # Click position within the pet.
            self.is_walking = False

    def mouseMoveEvent(self, event):
        if event.buttons() & Qt.LeftButton and self.drag_offset:
            global_pos = self.mapToGlobal(event.pos())
            new_x = global_pos.x() - self.drag_offset.x()
            new_y = global_pos.y() - self.drag_offset.y()

            # Calculates the velocity using the difference between positions.
            if self.last_mouse_pos:
                dx = global_pos.x() - self.last_mouse_pos.x()
                dy = global_pos.y() - self.last_mouse_pos.y()

                self.vx = math.floor(dx)
                self.vy = math.floor(dy)

            self.last_mouse_pos = global_pos

            # Updates the pet position to the mouse position.
            self.pos_x = new_x
            self.pos_y = new_y
            self.move(self.pos_x, self.pos_y)

    def mouseReleaseEvent(self, event):
        self.drag_offset = None
        self.last_mouse_pos = None

    
    def contextMenuEvent(self, event):
        self.context_menu.exec_(event.globalPos())

    # ========== Utility Functions ==========

    def reset_timer(self, callback, interval: int =15):
        """
        Resets the pet timer and connects a new callback.

        Parameters:
            callback: The new function that will be run every x seconds.
            interval (int): The interval (in milliseconds).
        """
        self.timer.stop()
        try:
            self.timer.timeout.disconnect()
        except TypeError:
            pass
    
        self.timer.timeout.connect(callback)
        self.timer.start(interval)

    def _start_delay(self):
        """
        Put the pet in pause mode.
        """

        self.on_delay = True
        delay_ms = random.randint(2000, 4000)  # between 2 and 4 seconds.
        self.delay_timer.start(delay_ms)

    def _end_delay(self):
        """
        Exit pause mode.
        """
        self.on_delay = False

    def show_info(self):
        """
        Shows the pet info.
        """
        self.info_window = InfoWindow(pet=self)
        self.info_window.show()

    def close_pet(self):
        """
        Close the pet window, destroy it and remove it from the manager's pet list.
        """
        self.manager.remove_pet(pet=self)
        self.close()      
        self.deleteLater()  

In [5]:
import time

class PetManager:
    def __init__(self):
        self.pets = [] 

        self.timer = QTimer()
        self.timer.timeout.connect(self.update)
        self.timer.start(50)  # 20 frames/second

    def add_pet(self, pet):
        """
        Adds a pet to the list of managed pets.

        Parameters:
            pet: object to be added to the list.
        """
        self.pets.append(pet)
        pet.manager = self

    def remove_pet(self, pet):
        """
        Removes a pet from the list of managed pets.

        Parameters:
            pet: object to be removed from the list.
        """
        if pet in self.pets:
            self.pets.remove(pet)

    def update(self):
        """
        Checks for interactions between the pets.
        """

        # Gets a list of only avaliable pets.
        active_pets = [p for p in self.pets if not p.in_battle]
        
        for i, pet1 in enumerate(active_pets):
            for pet2 in active_pets[i+1:]:

                # If all conditions are met, there is a random chance that a battle occurs.
                if self.check_proximity(pet1, pet2, proximity_x=100, proximity_y=100) and random.random() < 0.25:
                    self.battle(pet1, pet2)

    
    def check_proximity(self, pet1, pet2, proximity_x: int = 100, proximity_y: int = 100):
        """
        Checks if two pets are close, and facing each other.

        Parameters:
            pet1: First pet.
            pet2: Second pet.
            proximity_x (int): Maximum number of pixels in the x-axis for the interaction to occur.
            proximity_y (int): Maximum number of pixels in the y-axis for the interaction to occur.

        Returns:
            If all the conditions are met. (bool)
        """
        # Gets the distance between the pets.
        dx = abs(pet1.pos_x - pet2.pos_x)
        dy = abs(pet1.pos_y - pet2.pos_y)

        # Checks if they are facing each other.
        if pet1.pos_x < pet2.pos_x:
            facing_each_other = pet1.direction == 1 and pet2.direction == -1
        else:
            facing_each_other = pet1.direction == -1 and pet2.direction == 1
            
        return dx < proximity_x and dy < proximity_y and facing_each_other

    def battle(self, pet1, pet2):
        """
        Group the functions related to battle.

        Parameters:
            pet1: First pet.
            pet2: Second pet.
        """

        winner = self.resolve_battle(pet1, pet2)
        self.handle_battle_result(pet1, pet2, winner)

    def resolve_battle(self, pet1, pet2):
        """
        Decides the winner of the battle.

        Parameters:
            pet1: First pet.
            pet2: Second pet.

        Returns:
            The winner of the battle. (object)
        """

        n = 5
        # Two moves are chosen at random.
        move1 = random.randint(1, n)
        time.sleep(0.001)
        move2 = random.randint(1, n)

        # A rock-paper-scissor like script decides the winner.
        # If tie, the winner is None
        if move1 == move2:
            return None
        
        if (move1 + 1) % n == move2:
            return pet1
        elif (move2 + 1) % n == move1:
            return pet2
        else: # If an error happens
            return None 

    def handle_battle_result(self, pet1, pet2, winner):
        """
        Animates the pets according to the battle results.

        Parameters:
            pet1: First pet.
            pet2: Second pet.
            winner: Pet that won the battle.
        """
        iteration_counter = 0
        pet1.in_battle = True
        pet2.in_battle = True

        def animate():
            nonlocal iteration_counter
            iteration_counter += 1

            # If the pet is moved to somewhere distant from the other pet, the battle ends.
            if not self.check_proximity(pet1, pet2, proximity_x=200, proximity_y=200):
                pet1.in_battle = False
                pet2.in_battle = False
                timer.stop()
                return

            # Idle animation that the pets do when in battle.
            offset_y = 5 * math.sin(iteration_counter * 0.5)
            if pet1.is_walking:
                pet1.pos_y = math.floor(pet1.floor + offset_y)
            if pet2.is_walking:
                pet2.pos_y = math.floor(pet2.floor + offset_y)
            
            # When the battle is over
            if iteration_counter >= 500:
                timer.stop()
                
                if winner is pet1:
                    pet1.win_battle()
                    pet2.lose_battle()
                    pet1.level += 1
                    
                elif winner is pet2:
                    pet1.lose_battle()
                    pet2.win_battle()
                    pet2.level += 1

                else:
                    pet1.win_battle()
                    pet2.win_battle()
                
                pet1.in_battle = False
                pet2.in_battle = False

        # Starts the timer for the battle animations.
        timer = QTimer()
        timer.timeout.connect(animate)
        timer.start(15)

In [ ]:
# from Pet import Pet
from PyQt5.QtWidgets import QApplication
import sys
import os
import logging

def main():
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[logging.FileHandler("debug.log"), logging.StreamHandler()],
    )

    app = QApplication(sys.argv)
    app.setQuitOnLastWindowClosed(False)

    pet_names = set()
    for img in os.listdir("../imgs"):
        pet_name = img.split("_")[0]
        pet_names.add(pet_name)
    
    logging.info(f"Starting pets: {', '.join(pet_names)}")

    manager = PetManager()
    pets = []
    for name in pet_names:
        pets.append(Pet(name=name))
        pets[-1].show()

        manager.add_pet(pets[-1])
    app.exec_()

if __name__ == "__main__":
    main()


2025-11-29 20:06:15,070 [INFO] Starting pets: squirtle, charmander, bulbasaur
